## Парсер на основе библиотеки Scrapy

In [1]:
import platform
import scrapy
import logging
from scrapy.crawler import CrawlerProcess
import pandas as pd
from w3lib.html import strip_html5_whitespace as shw
from IPython.core.interactiveshell import InteractiveShell
import warnings
warnings.filterwarnings('ignore')
InteractiveShell.ast_node_interactivity = "all"

### Настройка пайплайнов
Класс `JsonWriterPipeline` создает пайплайн, в который пишет все найденные элементы в формат `JSON`.
Все данные будут сохраняться в два файл: первый с расширением `.json`, второй с расширением `.jl` (в нем каждый элемент json находится на новой странице).

In [2]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('data/habr/habr_data.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

### Создание класса для парсинга
В списке `start_urls` генерируются URL страниц, из которых нужно извлечь информацию: 
- `time` - время публикации статьи
- `tags` - теги статьи
- `habs` - хабы (тематика)
- `saved` - количество сохранений

In [3]:
class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'https://habr.com/ru/post/{0}/'.format(i) for i in range(450000, 450100)
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},
        'FEED_FORMAT':'json',                               
        'FEED_URI': 'data/habr/habr_data.json'                      
    }
    
    def parse(self, response):
        yield {
            'time': shw(response.xpath("//span[@class='tm-article-snippet__datetime-published']/time/@datetime").get()),
            'tags': [shw(result.lower()) 
                         for result in response.xpath("//a[@class='tm-tags-list__link']/text()").getall()],
            'habs': [shw(result.lower()) 
                         for result in response.xpath("//a[@class='tm-hubs-list__link']/text()").getall()],
            'saved': shw(response.xpath("//span[@class='bookmarks-button__counter']/text()").get()),
            }
        

### Парсинг

In [4]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

2021-10-17 19:37:45 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2021-10-17 19:37:45 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.7.6 (default, Jan  8 2020, 13:42:34) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Darwin-20.2.0-x86_64-i386-64bit
2021-10-17 19:37:45 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-10-17 19:37:45 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


### Обзор полученных данных

In [5]:
habr_dataset = pd.read_json('data/habr/habr_data.json')
habr_dataset.head(10)

,time,tags,habs,saved
0,2019-04-29T13:48:10.000Z,"[lru, mru, cache]","[высокая производительность, программирование,...",100
1,2019-04-29T13:43:19.000Z,"[арабский язык, ltr, rtl, unicode]","[разработка веб-сайтов, типографика, дизайн]",21
2,2019-04-29T15:55:25.000Z,"[фотография, резервное копирование, облако, li...","[резервное копирование, хранение данных, фотот...",157
3,2019-04-29T14:32:29.000Z,"[формальная верификация, формальные методы, ре...","[децентрализованные сети, информационная безоп...",46
4,2019-04-29T13:57:01.000Z,"[охлаждение цод, цод, чиллерное охлаждение]","[хостинг, it-инфраструктура, it-компании, инже...",7
5,2019-04-30T07:00:03.000Z,"[гироид, минимальные поверхности]","[математика, научно-популярное]",46
6,2019-04-30T05:15:04.000Z,"[сша, образование, summit learning]","[образование за рубежом, будущее здесь]",43
7,2019-04-29T13:43:52.000Z,"[c++, clang, llvm, compiler, pvs-studio, open ...","[open source, c++, компиляторы, devops]",1
8,2019-04-29T14:21:44.000Z,"[россвязь, чс, оповещения, законодательство]","[разработка систем связи, законодательство в it]",2
9,2019-04-30T12:00:01.000Z,"[oculus, rift s, шлем виртуальной реальности]",[ar и vr],6


In [6]:
habr_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   time    48 non-null     object
 1   tags    48 non-null     object
 2   habs    48 non-null     object
 3   saved   48 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 1.6+ KB


Получилось 48 непустых объектов (было сгенерировано 100 страниц, но так как некоторые посты были удалены, информация с них не была скачена)